# Usage of Deeplab + LIP model and extracion of ROI 

In this notebook, the segmentation of the desired images are done, followed by a contour processing to reduce the impact of noise and segmentation errors.

## Dependencies

In [ ]:
import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib
import cv2
import glob

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
from IPython.display import display
from google.colab.patches import cv2_imshow
import pandas as pd

%tensorflow_version 1.x
import tensorflow as tf

import ipywidgets as widgets
import math

TensorFlow 1.x selected.


## Path definitions - change to your own

In [ ]:
MODEL_INFER_GRAPH = "/content/drive/MyDrive/TFM/model_19_60k/frozen_inference_graph.pb"
VIS_IMAGES = "/content/drive/MyDrive/TFM/CROP3/"
EXPORT_CROP = "/content/drive/MyDrive/TFM/SegResults9"

USING_ANNOTATIONS = True
VIS_IMAGES_ANNOTATIONS = "/content/drive/MyDrive/TFM/CROP3/annotateTest.csv" 
ANNOTATION_COLUMNS = ["filename", "x_min", "y_min", "x_max", "y_max", "class"]
NEW_ANNOTATIONS = pd.DataFrame(columns=ANNOTATION_COLUMNS)

## Mount Google Drive storage
In case you need it

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Deeplab model class to perform the segmentation and visualize it

In [ ]:
class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, frozen_graph):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    f = open(frozen_graph, "rb")
    graph_df = tf.GraphDef.FromString(f.read())

    if not os.path.isfile(frozen_graph) or graph_df is None:
      raise RuntimeError('Cannot find inference graph.')

    with self.graph.as_default():
      tf.import_graph_def(graph_df, name='')

    self.sess = tf.Session(graph=self.graph)

  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(image)]}
    )
    
    seg_map = unify_labels(batch_seg_map[0])
    

    return image, seg_map


def unify_labels(seg_map):
  """Unifies certains labels from LIP in the segmentation map.

    Args:
      seg_map: Segmentation map from DeeplabModel run(sel, image) method

    Returns:
      seg_map: Segmentation map with unified labels.
    """

  # Dress and Coat will be appear as UpperClothes
  seg_map = np.where(seg_map==6, 5, seg_map) # UpperClothes == Dress
  seg_map = np.where(seg_map==7, 5, seg_map) # UpperClothes == Coat
  
  # Fuse left and right arm in a single instance
  seg_map = np.where(seg_map==14, 15, seg_map) 

  # Fuse left and right leg in a single instance
  seg_map = np.where(seg_map==16, 17, seg_map)

  # Fuse left and right shoe in a single instance
  seg_map = np.where(seg_map==18, 19, seg_map) 

  return seg_map

def create_pascal_label_colormap():
  """Creates a label colormap used in PASCAL VOC segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
  colormap = np.zeros((256, 3), dtype=int)
  ind = np.arange(256, dtype=int)

  for shift in reversed(range(8)):
    for channel in range(3):
      colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

  return colormap


def label_to_color_image(label):
  """Adds color defined by the dataset colormap to the label.

  Args:
    label: A 2D array with integer type, storing the segmentation label.

  Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

  Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
  """
  if label.ndim != 2:
    raise ValueError('Expect 2-D input label')

  colormap = create_pascal_label_colormap()

  if np.max(label) >= len(colormap):
    raise ValueError('label value too large.')

  return colormap[label]


def vis_segmentation(image, seg_map, body_color, pants_color):
  """Visualizes input image, segmentation map, segmentation overlay, blob overlay and biggest blob overlay view."""
  plt.figure(figsize=(30, 5))
  grid_spec = gridspec.GridSpec(1, 6, width_ratios=[6, 6, 6, 6, 6, 1])

  plt.subplot(grid_spec[0])
  plt.imshow(image)
  plt.axis('off')
  plt.title('input image')

  plt.subplot(grid_spec[1])
  seg_image = label_to_color_image(seg_map).astype(np.uint8)
  plt.imshow(seg_image)
  plt.axis('off')
  plt.title('segmentation map')

  plt.subplot(grid_spec[2])
  plt.imshow(image)
  plt.imshow(seg_image, alpha=0.7)
  plt.axis('off')
  plt.title('segmentation overlay')

  plt.subplot(grid_spec[3])
  blob_image, biggest_blobs, _ = get_contours_with_colors(seg_image, body_color, pants_color)
  plt.imshow(image)
  plt.imshow(blob_image, alpha=0.7)
  plt.axis('off')
  plt.title('blob overlay')

  plt.subplot(grid_spec[4])
  plt.imshow(image)
  plt.imshow(biggest_blobs, alpha=0.7)
  plt.axis('off')
  plt.title('biggest blob overlay')

  unique_labels = np.unique(seg_map)
  ax = plt.subplot(grid_spec[5])
  plt.imshow(FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
  ax.yaxis.tick_right()
  plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
  plt.xticks([], [])
  ax.tick_params(width=0.0)
  plt.grid('off')
  plt.show()


LABEL_NAMES = np.asarray([
    'Background', 'Hat', 'Hair', 'Glove', 'Sunglasses', 'UpperClothes', 'Dress', 'Coat',
    'Socks', 'Pants', 'Jumpsuits', 'Scarf', 'Skirt', 'Face', 'Left-Arm', 'Arm',
    'Left-leg', 'Leg', 'Left-shoe', 'Shoe'
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

MODEL = DeepLabModel(MODEL_INFER_GRAPH)

In [ ]:
BODY_COLOR = FULL_COLOR_MAP[5]
PANTS_COLOR = FULL_COLOR_MAP[9]

## Functions definitions

In [ ]:
def save_annotation(original_annotations, object_detection):
  global NEW_ANNOTATIONS

  if NEW_ANNOTATIONS is not None and original_annotations is not None:
    df_ann = pd.read_csv(original_annotations, header=None)
    df_ann.columns = ["filename", "x_min", "y_min", "x_max", "y_max", "class"]
    df_ann = df_ann[df_ann['filename'].str.endswith(os.path.basename(object_detection['filename']))]

    for index, row in df_ann.iterrows():
      x_min = row['x_min'] - object_detection['x']
      y_min = row['y_min'] - object_detection['y']
      x_max = (object_detection['x2'] - object_detection['x']) - (object_detection['x2'] -  row['x_max'])
      y_max = (object_detection['y2'] - object_detection['y']) - (object_detection['y2'] -  row['y_max'])

      if (x_min >= 0 and y_min >= 0 and x_max >= 0 and y_max >= 0) and (x_min <= (object_detection['x2'] - object_detection['x']) and y_min <= (object_detection['y2'] - object_detection['y']) and x_max <= (object_detection['x2'] - object_detection['x']) and y_max <= (object_detection['y2'] - object_detection['y'])):
        NEW_ANNOTATIONS = NEW_ANNOTATIONS.append({
            'filename': object_detection['filename'],
            'x_min': x_min,
            'y_min': y_min,
            'x_max': x_max,
            'y_max': y_max,
            'class': row['class']
        }, ignore_index=True)
        print(NEW_ANNOTATIONS.tail().iloc[:, 1:5])
      else:
        print('Not valid annotation placing')

In [ ]:
def biggest_contours(contours_body, contours_pants):
  """Finds the contours with highest area with body and pants label.

    Args:
      contours_body: Contours found labeled as body
      contours_pants: Contours found labeled as pants

    Returns:
      biggest_body: Index of the highest-area body contour
      biggest_pants: Index of the highest-area pants contour
  """  
  pants_centroid = None
  biggest_pants = 0

  for index, pants in enumerate(contours_pants):
    if cv2.contourArea(pants) > cv2.contourArea(contours_pants[biggest_pants]):
      M = cv2.moments(pants)
      cx = int(M["m10"] / M["m00"]) if int(M["m00"]) > 0 else 0
      cy = int(M["m01"] / M["m00"]) if int(M["m00"]) > 0 else 0
      pants_centroid = (cx, cy)
      biggest_pants = index

  biggest_body = None

  for index, body in enumerate(contours_body):
    M = cv2.moments(body)

    cx = int(M["m10"] / M["m00"]) if int(M["m00"]) > 0 else 0
    cy = int(M["m01"] / M["m00"]) if int(M["m00"]) > 0 else 0
    body_centroid = (cx, cy)

    if body_centroid[0] == 0 and body_centroid[1] == 0:
      continue

    if pants_centroid != None and body_centroid[1] < pants_centroid[1]:
      if biggest_body == None or cv2.contourArea(body) > cv2.contourArea(contours_body[biggest_body]):
        biggest_body = index
    
    if pants_centroid == None:
       if biggest_body == None or cv2.contourArea(body) > cv2.contourArea(contours_body[biggest_body]):
        biggest_body = index

  return biggest_body, biggest_pants

In [ ]:
def crop_roi(im, seg_map, body_color, pants_color):
  """Crops the image to the ROI (body + pants).

    Args:
      im: Image to crop
      seg_map: Segmentation map of the image to crop
      body_color: Body color in the segmentation map
      pants_color: Pants color in the segmentation map

    Returns:
      out: Cropped image within the ROI
  """
  img = cv2.imread(im)

  seg_image = label_to_color_image(seg_map).astype(np.uint8)

  _, _, contours = get_contours_with_colors(seg_image, body_color, pants_color)
  i_body, i_pants = biggest_contours(contours[0], contours[1])

  out = img

  height, width = img.shape[:2]

  if contours[0] !=  None and len(contours) > 0 and len(contours[0]) > 0 and i_body != None:
    xb, yb, wb, hb = cv2.boundingRect(contours[0][i_body])

    # If no pants are found, upper bound should be calculated with the body
    xp = xb
    yp = yb
    wp = wb
    hp = hb

    if contours[1] !=  None and len(contours) > 1 and len(contours[1]) > 1:
      xp, yp, wp, hp = cv2.boundingRect(contours[1][i_pants])

    mask = np.zeros_like(img)

    cv2.drawContours(mask, contours[0], i_body, (255,255,255), -1)
    if contours[1] !=  None and len(contours) > 1 and len(contours[1]) > 1:
      cv2.drawContours(mask, contours[1], i_pants, (255,255,255), -1)

    out = np.zeros_like(img)

    out[mask == 255] = img[mask == 255]
    out[mask == 0] = img[mask == 0]

    (top_y, top_x) = (yb, min(xb, xp))
    (bottom_y, bottom_x) = (yp + hp, max(xb + wb, xp + wp))
    out = out[top_y:bottom_y+1, top_x:bottom_x+1]

    if USING_ANNOTATIONS:
      save_annotation(VIS_IMAGES_ANNOTATIONS, {
          'filename': im, 
          'x': top_x, 
          'y': top_y, 
          'x2': bottom_x, 
          'y2': bottom_y
          }) 
  else:
    # Upper body segmentation not found: Save the annotation like the original
    if USING_ANNOTATIONS:
      print('Body segmentation not found')
      save_annotation(VIS_IMAGES_ANNOTATIONS, {
          'filename': im, 
          'x': 0, 
          'y': 0, 
          'x2': width, 
          'y2': height
          }) 

  return cv2.cvtColor(out, cv2.COLOR_BGR2RGB)

In [ ]:
def get_contours_with_colors(seg_map, body_color, pants_color):
  """Get contours of the ROI (body + pants).

    Args:
      seg_map: Segmentation map of the image
      body_color: Body color in the segmentation map
      pants_color: Pants color in the segmentation map

    Returns:
      seg_map: Segmentation map with the contours drawn
      seg_map_biggest_blobs: Segmentation map with the highest-area contours drawn
      [contours_body, contours_pants]: List of the contours found
  """ 
  seg_map_body = np.copy(seg_map)
  seg_map_pants = np.copy(seg_map)
  seg_map_biggest_blobs = np.copy(seg_map)

  seg_map_body[seg_map_body == body_color] = 255
  seg_map_pants[seg_map_pants == pants_color] = 255

  imgray_body = cv2.cvtColor(seg_map_body, cv2.COLOR_BGR2GRAY)
  imgray_pants = cv2.cvtColor(seg_map_pants, cv2.COLOR_BGR2GRAY)

  _, thresh_body = cv2.threshold(imgray_body, 254, 255, cv2.THRESH_BINARY)
  _, thresh_pants = cv2.threshold(imgray_pants, 254, 255, cv2.THRESH_BINARY)

  contours_body, _ = cv2.findContours(thresh_body, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contours_pants, _ = cv2.findContours(thresh_pants, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  biggest_body, biggest_pants = biggest_contours(contours_body, contours_pants)

  cv2.drawContours(seg_map, contours_body, -1, (0,255,0), 3)
  cv2.drawContours(seg_map, contours_pants, -1, (255,255,0), 3)

  if biggest_body != None:
    cv2.drawContours(seg_map_biggest_blobs, contours_body, biggest_body, (0,255,0), 3)
  if biggest_pants != None:
    cv2.drawContours(seg_map_biggest_blobs, contours_pants, biggest_pants, (255,255,0), 3)

  return seg_map, seg_map_biggest_blobs, [contours_body, contours_pants]

## Visualize results

In [ ]:
image_list_siani = os.listdir(VIS_IMAGES)
image_picker_siani = widgets.Dropdown(options=image_list_siani, value=image_list_siani[0], disabled=False)
image_picker_siani

Dropdown(options=('499_Arucas_frame_01_36_03_000.jpg', '500_Arucas_frame_01_36_03_000.jpg', '501_Arucas_frame_…

### Segmentation and blobs

In [ ]:
def run_visualization_siani(file):
  """Inferences DeepLab model and visualizes result."""
  try:
    original_im = Image.open(os.path.join(VIS_IMAGES, file))
  except IOError:
    print('Cannot retrieve image. Please check file: ' + os.path.join(VIS_IMAGES, file))
    return

  print('running deeplab on image %s...' % file)
  resized_im, seg_map = MODEL.run(original_im)

  vis_segmentation(resized_im, seg_map, BODY_COLOR, PANTS_COLOR)

In [ ]:
run_visualization_siani(image_picker_siani.value)

### Cropping

In [ ]:
def run_crop_siani(file):
  """Inferences DeepLab model and visualizes result."""
  try:
    original_im = Image.open(os.path.join(VIS_IMAGES, file))
  except IOError:
    print('Cannot retrieve image. Please check file: ' + os.path.join(VIS_IMAGES, file))
    return

  print('running deeplab on image %s...' % file)
  resized_im, seg_map = MODEL.run(original_im)

  return resized_im, crop_roi(os.path.join(VIS_IMAGES, file), seg_map, BODY_COLOR, PANTS_COLOR)

In [ ]:
img, crop = run_crop_siani(image_picker_siani.value)

print(f'\nOriginal image: ({img.size[1]}, {img.size[0]})')
print(f'Cropped image: {crop.shape}\n')

fig = plt.figure(figsize=(10, 10))
fig.add_subplot(1,2,1)
plt.imshow(img)
plt.axis('off')
fig.add_subplot(1,2,2)
plt.imshow(Image.fromarray(crop))
plt.axis('off')

## Crop and save all images in a folder

In [ ]:
%mkdir {EXPORT_CROP}

### Format: A single directory with all the images

In [ ]:
total = len(glob.glob1(VIS_IMAGES, "*.jpg"))
count = 0

for filename in glob.glob(VIS_IMAGES + '/*.jpg'):
  _, crop = run_crop_siani(os.path.join(VIS_IMAGES, filename))
  cv2.imwrite(os.path.join(EXPORT_CROP, os.path.basename(filename)), cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))
  if USING_ANNOTATIONS and NEW_ANNOTATIONS is not None:
    NEW_ANNOTATIONS.to_csv(os.path.join(EXPORT_CROP, 'annotateTest.csv'), header=None, index=False)
  count = count + 1
  print(f'Image {count}/{total}')

Se han truncado las últimas 5000 líneas del flujo de salida.
22    42   119    92   153
23   113   199   215   273
24    49   143    92   184
25    68   173   142   234
26    57   157   135   211
Image 108/1498
running deeplab on image /content/drive/MyDrive/TFM/CROP3/607_Arucas_frame_01_52_20_000.jpg...
   x_min y_min x_max y_max
23   113   199   215   273
24    49   143    92   184
25    68   173   142   234
26    57   157   135   211
27   208   527   375   654
Not valid annotation placing
Image 109/1498
running deeplab on image /content/drive/MyDrive/TFM/CROP3/608_Arucas_frame_01_52_20_000.jpg...
Image 110/1498
running deeplab on image /content/drive/MyDrive/TFM/CROP3/609_Arucas_frame_01_52_20_000.jpg...
Image 111/1498
running deeplab on image /content/drive/MyDrive/TFM/CROP3/610_Arucas_frame_01_39_44_000.jpg...
   x_min y_min x_max y_max
24    49   143    92   184
25    68   173   142   234
26    57   157   135   211
27   208   527   375   654
28   105   183   225   264
Image 112/1

### Format 2: A directory with multiple directories with images in each of them

In [ ]:
count = 0

for root, subdirs, files in os.walk(VIS_IMAGES):
  if len(subdirs) == 0:
    dir = os.path.join(EXPORT_CROP, os.path.basename(root))
    try:
      os.mkdir(dir)
      print(f'[+] Directory {dir} created') 
    except FileExistsError:
      pass

    for file in files:
      file_path = os.path.join(os.path.join(VIS_IMAGES, os.path.basename(root)), file)
      _, crop = run_crop_siani(file_path)
      cv2.imwrite(os.path.join(dir, file), cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))
      count = count + 1
      print(f'Image {count}')